In [2]:
import pandas as pd
import numpy as np
import torch
import random
import os
import sys

# ==========================================
# 1. 라이브러리 및 환경 설정
# ==========================================
try:
    from sklearn.metrics import classification_report, accuracy_score, f1_score
    from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding, BigBirdConfig
    from datasets import Dataset
    from sklearn.model_selection import train_test_split
except ImportError:
    sys.exit("❌ 라이브러리가 설치되지 않았습니다. !pip install transformers datasets accelerate scikit-learn 실행 필요")

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🚀 [KoBigBird] 사용 장치: {device}")

# ==========================================
# 2. 데이터 로드 및 병합
# ==========================================
url = "https://raw.githubusercontent.com/tunib-ai/DKTC/main/data/train.csv"
try:
    df_threat = pd.read_csv(url)[['class', 'conversation']]
    print(f"✅ 위협 데이터 로드 완료: {len(df_threat)}개")
except Exception as e:
    sys.exit(f"❌ 위협 데이터 로드 실패: {e}")

# 일반 대화 파일 로드 (파일명 확인)
normal_file = "/content/drive/MyDrive/DLThon/normal_conv/normal_conversation.csv"
if not os.path.exists(normal_file):
    if os.path.exists("normal_conversation.csv"):
        normal_file = "normal_conversation.csv"
    else:
        sys.exit(f"❌ '{normal_file}' 파일을 찾을 수 없습니다.")

print(f"📂 사용할 일반 대화 파일: {normal_file}")
df_normal = pd.read_csv(normal_file)
df_normal['conversation'] = df_normal['conversation'].str.replace(r'(^|\n)[AB]:\s*', '', regex=True)
if 'class' not in df_normal.columns:
    df_normal['class'] = '일반 대화'
df_normal = df_normal[['class', 'conversation']]
print(f"✅ 일반 대화 데이터 로드 완료: {len(df_normal)}개")

# 병합
df_final = pd.concat([df_threat, df_normal], ignore_index=True)
df_final = df_final.sample(frac=1, random_state=42).reset_index(drop=True)


import pandas as pd
import numpy as np
import torch
from transformers import MarianMTModel, MarianTokenizer
from tqdm import tqdm
import random

# ==========================================
# 1. 역번역(Back-Translation) 모델 설정
# ==========================================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# ---------------------------------------------------------
# [수정 1] 한국어 -> 영어 모델 (기존 동일)
# ---------------------------------------------------------
ko_en_model_name = 'Helsinki-NLP/opus-mt-ko-en'
try:
    ko_en_tokenizer = MarianTokenizer.from_pretrained(ko_en_model_name)
    ko_en_model = MarianMTModel.from_pretrained(ko_en_model_name).to(device)
except Exception as e:
    print(f"Ko->En 모델 로드 실패: {e}")

# ---------------------------------------------------------
# [수정 2] 영어 -> 한국어 모델 (이름 변경!)
# opus-mt-en-ko (X) -> opus-mt-tc-big-en-ko (O)
# ---------------------------------------------------------
en_ko_model_name = 'Helsinki-NLP/opus-mt-tc-big-en-ko'
try:
    en_ko_tokenizer = MarianTokenizer.from_pretrained(en_ko_model_name)
    en_ko_model = MarianMTModel.from_pretrained(en_ko_model_name).to(device)
    print("✅ 번역 모델 로드 완료 (Model Name Corrected)")
except Exception as e:
    print(f"En->Ko 모델 로드 실패: {e}")
    print("오류가 지속되면 'pip install sentencepiece'를 실행했는지 확인해주세요.")

def back_translate(texts, batch_size=32):
    """
    한국어 문장 리스트를 입력받아 역번역(KO->EN->KO)된 리스트를 반환합니다.
    """
    augmented_texts = []

    # 배치 단위 처리
    for i in tqdm(range(0, len(texts), batch_size), desc="Back-Translation"):
        batch = texts[i:i+batch_size]

        try:
            # 1. KR -> EN
            tokens = ko_en_tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
            translated = ko_en_model.generate(**tokens)
            en_texts = [ko_en_tokenizer.decode(t, skip_special_tokens=True) for t in translated]

            # 2. EN -> KR
            tokens = en_ko_tokenizer(en_texts, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
            translated = en_ko_model.generate(**tokens)
            ko_texts = [en_ko_tokenizer.decode(t, skip_special_tokens=True) for t in translated]

            augmented_texts.extend(ko_texts)
        except Exception as e:
            print(f"Batch Processing Error: {e}")
            # 에러 발생 시 원본 반환 (중단 방지)
            augmented_texts.extend(batch)

    return augmented_texts

# ==========================================
# 2. 데이터 증강 적용 실행
# ==========================================
# df_threat는 로드되어 있다고 가정
target_classes = ['협박 대화', '갈취 대화', '직장 내 괴롭힘 대화', '기타 괴롭힘 대화']
augmented_data = []

print("🚀 데이터 증강 시작")

for cls in target_classes:
    cls_data = df_threat[df_threat['class'] == cls]
    original_texts = cls_data['conversation'].tolist()

    print(f"\nProcessing class: {cls} (Count: {len(original_texts)})")

    # 시간 절약을 위해 클래스당 200개만 샘플링하여 테스트 (전체 하려면 이 줄 주석 처리)
    # original_texts = original_texts[:200]

    # 역번역 실행
    augmented_texts_bt = back_translate(original_texts, batch_size=8) # OOM 방지 위해 배치 8

    for text in augmented_texts_bt:
        augmented_data.append({'class': cls, 'conversation': text})

# 결과 확인
df_augmented = pd.DataFrame(augmented_data)
print(f"\n✅ 증강 완료! 추가된 데이터 수: {len(df_augmented)}")
print(df_augmented.head())

# [중요] 이후 병합 코드
# df_final_aug = pd.concat([df_threat, df_augmented, df_normal], ignore_index=True)

# ==========================================
# 3. 라벨 인코딩 & 데이터셋 변환
# ==========================================
label_map = {
    '협박 대화': 0, '갈취 대화': 1, '직장 내 괴롭힘 대화': 2, '기타 괴롭힘 대화': 3,
    '협박': 0, '갈취': 1, '직장 내 괴롭힘': 2, '기타 괴롭힘': 3,
    '일반 대화': 4
}
df_final['label'] = df_final['class'].map(label_map)
df_final = df_final.dropna(subset=['label'])
df_final['label'] = df_final['label'].astype(int)

train_df, val_df = train_test_split(df_final, test_size=0.2, random_state=42, stratify=df_final['label'])
train_dataset = Dataset.from_pandas(train_df[['conversation', 'label']])
val_dataset = Dataset.from_pandas(val_df[['conversation', 'label']])

# ==========================================
# 4. 모델 & 토크나이저 (KoBigBird)
# ==========================================
# 🚨 [변경됨] KoBigBird 모델 사용
MODEL_NAME = "monologg/kobigbird-bert-base"
print(f"🔄 모델 로딩 중: {MODEL_NAME}")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# 🚨 [중요] BigBird 설정: 짧은 문장(4096 토큰 미만) 처리 시 original_full 사용 권장
# 만약 메모리가 부족하면 이 부분을 삭제하여 기본값(block_sparse)으로 사용하세요.
config = BigBirdConfig.from_pretrained(MODEL_NAME)
config.attention_type = "original_full"
config.num_labels = 5

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, config=config).to(device)

def preprocess_function(examples):
    # BigBird는 긴 문맥(최대 4096) 처리가 장점이지만,
    # 이번 데이터셋은 대화가 그렇게 길지 않으므로 512~1024 정도로 설정해도 충분합니다.
    # 여기서는 안전하게 512로 설정합니다. (필요 시 1024로 늘리세요)
    return tokenizer(examples["conversation"], truncation=True, max_length=512)

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# ==========================================
# 5. 학습 (Training)
# ==========================================
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    return {"accuracy": acc, "f1": f1}

training_args = TrainingArguments(
    output_dir="./results_bigbird",  # [변경됨] 저장 폴더
    learning_rate=2e-5,
    per_device_train_batch_size=8,   # [변경됨] 메모리 절약을 위해 8로 조정 (가능하면 16)
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,   # 배치 8 * 2 = 실제 16 효과
    num_train_epochs=5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

print("\n🦅 KoBigBird 학습 시작! (시간이 조금 더 걸릴 수 있습니다)")
trainer.train()

# ==========================================
# 6. 모델 저장
# ==========================================
SAVE_PATH = "./final_model_bigbird" # [변경됨] 저장 폴더
model.save_pretrained(SAVE_PATH)
tokenizer.save_pretrained(SAVE_PATH)
print(f"\n✅ 학습 완료! 모델이 '{SAVE_PATH}' 폴더에 저장되었습니다.")

🚀 [KoBigBird] 사용 장치: cuda
✅ 위협 데이터 로드 완료: 3950개
📂 사용할 일반 대화 파일: /content/drive/MyDrive/DLThon/normal_conv/normal_conversation.csv
✅ 일반 대화 데이터 로드 완료: 800개
Using device: cuda


/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:176: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Loading weights:   0%|          | 0/258 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie model.shared.weight to model.decoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


tokenizer_config.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['bert.embeddings.LayerNorm.weight', 'bert.embeddings.LayerNorm.bias', 'bert.encoder.layer.0.attention.output.LayerNorm.weight', 'bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.encoder.layer.0.output.LayerNorm.weight', 'bert.encoder.layer.0.output.LayerNorm.bias', 'bert.encoder.layer.1.attention.output.LayerNorm.weight', 'bert.encoder.layer.1.attention.output.LayerNorm.bias', 'bert.encoder.layer.1.output.LayerNorm.weight', 'bert.encoder.layer.1.output.LayerNorm.bias', 'bert.encoder.layer.2.attention.output.LayerNorm.weight', 'bert.encoder.layer.2.attention.output.LayerNorm.bias', 'bert.encoder.layer.2.output.LayerNorm.weight', 'bert.encoder.layer.2.output.LayerNorm.bias', 'bert.encoder.layer.3.attention.output.LayerNorm.weight', 'bert.encoder.layer.3.attention.output.LayerNorm.bias', 'bert.encoder.layer.3.output.LayerNorm.weight', 'bert.encoder.layer.3.output.LayerNorm.bias', 'bert.encoder.layer.4.attention.output.La

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]


✅ 학습 완료! 모델이 './final_model_bigbird' 폴더에 저장되었습니다.


In [4]:
import pandas as pd
import numpy as np
import torch
from sklearn.metrics import classification_report, accuracy_score, f1_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# ==========================================
# 1. 설정 및 데이터 준비
# ==========================================
MODEL_PATH = "./final_model_bigbird"       # 학습된 모델 경로
DATA_PATH = "/content/drive/MyDrive/DLThon/data/train.csv" # 데이터 파일 경로

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 데이터 로드
df = pd.read_csv(DATA_PATH)
df['conversation'] = df['conversation'].astype(str)

# 라벨 재매핑 (안전장치)
label_map = {
    '협박 대화': 0, '갈취 대화': 1, '직장 내 괴롭힘 대화': 2, '기타 괴롭힘 대화': 3,
    '협박': 0, '갈취': 1, '직장 내 괴롭힘': 2, '기타 괴롭힘': 3,
    '직장 괴롭힘': 2, '기타 괴롭힘': 3,
    '일반 대화': 4
}
df['label'] = df['class'].map(label_map)

# 결측치 제거
df = df.dropna(subset=['label'])
df['label'] = df['label'].astype(int)

# ---------------------------------------------------------
# ⚖️ [핵심] 클래스별 100개씩 균형 샘플링
# ---------------------------------------------------------
# 각 라벨별로 랜덤하게 100개씩 뽑습니다.
try:
    test_df = df.groupby('label').apply(lambda x: x.sample(n=100, random_state=42)).reset_index(drop=True)
    print(f"테스트 데이터셋 구성 완료: 총 {len(test_df)}개")
    print(test_df['class'].value_counts()) # 각 100개인지 확인
except ValueError as e:
    print(f"오류: 데이터가 부족하여 클래스별 100개를 뽑을 수 없습니다. ({e})")
    # 예외 시 전체 데이터 사용
    test_df = df

# ==========================================
# 2. 모델 로드 및 예측
# ==========================================
print("\n모델 로딩 중...")
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH).to(device)
    model.eval()
except OSError:
    print("오류: 저장된 모델을 찾을 수 없습니다. 학습 코드를 먼저 실행했는지 확인해주세요.")
    # (코드가 멈추지 않도록 임시 종료 처리 필요 시 exit())

# 예측 함수
def predict_batch(texts, batch_size=32):
    all_preds = []
    # 데이터가 많을 경우 배치를 나눠서 처리
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors="pt", truncation=True, padding=True, max_length=256).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
        preds = torch.argmax(logits, dim=-1).cpu().numpy()
        all_preds.extend(preds)
    return np.array(all_preds)

print("예측 시작... (잠시만 기다려주세요)")
y_true = test_df['label'].tolist()
y_pred = predict_batch(test_df['conversation'].tolist())

# ==========================================
# 3. 평가 지표 출력 (Text Only)
# ==========================================
target_names = ['협박', '갈취', '직장 괴롭힘', '기타 괴롭힘', '일반 대화']

print("\n" + "="*50)
print("[최종 모델 평가 리포트 (Test Set: 100 samples/class)]")
print("="*50)

# 1. Accuracy & F1-Score
acc = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average='weighted') # 각 클래스 비중이 같으므로 macro와 유사함

print(f"정확도 (Accuracy): {acc:.4f}")
print(f"F1 점수 (Weighted): {f1:.4f}")
print("-" * 50)

# 2. Classification Report (상세 지표)
print("\n[클래스별 상세 지표]")
# digits=4 옵션으로 소수점 4자리까지 정밀하게 출력
print(classification_report(y_true, y_pred, target_names=target_names, digits=4))
print("="*50)

/tmp/ipython-input-1532636050.py:38: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_df = df.groupby('label').apply(lambda x: x.sample(n=100, random_state=42)).reset_index(drop=True)


테스트 데이터셋 구성 완료: 총 400개
class
협박 대화          100
갈취 대화          100
직장 내 괴롭힘 대화    100
기타 괴롭힘 대화      100
Name: count, dtype: int64

모델 로딩 중...


Loading weights:   0%|          | 0/203 [00:00<?, ?it/s]

예측 시작... (잠시만 기다려주세요)

[최종 모델 평가 리포트 (Test Set: 100 samples/class)]
정확도 (Accuracy): 0.9525
F1 점수 (Weighted): 0.9534
--------------------------------------------------

[클래스별 상세 지표]
              precision    recall  f1-score   support

          협박     0.9596    0.9500    0.9548       100
          갈취     0.9412    0.9600    0.9505       100
      직장 괴롭힘     0.9615    1.0000    0.9804       100
      기타 괴롭힘     0.9574    0.9000    0.9278       100
       일반 대화     0.0000    0.0000    0.0000         0

    accuracy                         0.9525       400
   macro avg     0.7640    0.7620    0.7627       400
weighted avg     0.9549    0.9525    0.9534       400



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [7]:
import pandas as pd
import json
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm


# 1. 설정 및 데이터 로드
MODEL_PATH = "./final_model_bigbird"       # 학습된 모델 경로
TEST_FILE = "/content/test (1).json"            # 업로드된 테스트 파일

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 장치: {device}")

# 테스트 데이터 로드 (JSON)
try:
    with open(TEST_FILE, 'r', encoding='utf-8') as f:
        test_data = json.load(f)
    print(f"테스트 데이터 로드 완료: {len(test_data)}개")

    # 데이터 구조 확인 (첫 번째 아이템)
    first_key = list(test_data.keys())[0]
    print(f"데이터 예시: ID={first_key}, 내용={test_data[first_key]}")

except FileNotFoundError:
    print("오류: 'test.json' 파일을 찾을 수 없습니다.")
    # (테스트용 더미 데이터)
    test_data = {"t_000": {"text": "이거 들어봐 와 이 노래 진짜 좋다"}, "t_001": {"text": "야 돈 내놔"}}


# 2. 모델 로드
print("모델 로딩 중...")
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH).to(device)
    model.eval()
except OSError:
    print("오류: 저장된 모델을 찾을 수 없습니다. 학습 코드를 먼저 실행했는지 확인해주세요.")
    # (임시 모델 로드)
    tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
    model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base", num_labels=5).to(device)

# 3. 추론 (Inference)
print("예측 시작...")

results = []

# test_data가 딕셔너리 형태 {"t_000": {"text": "..."}} 라고 가정
for idx, item in tqdm(test_data.items()):
    # 텍스트 추출 (text 키가 없으면 conversation 등 다른 키 시도)
    text = item.get('text', item.get('conversation', ''))

    # 전처리 및 토크나이징
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # 예측값 (0~4)
    pred_label = torch.argmax(logits, dim=-1).item()

    # 결과 저장
    results.append({
        'idx': idx,
        'class': pred_label  # 숫자로 저장 (0, 1, 2, 3, 4)
    })

# 4. 제출 파일 생성
submission = pd.DataFrame(results)

# 컬럼 순서 정렬 (idx, class)
submission = submission[['idx', 'class']]

# 파일 저장
save_path = "submission.csv"
submission.to_csv(save_path, index=False)

print("\n" + "="*50)
print(f"제출 파일 생성 완료: {save_path}")
print("="*50)
print(submission.head())

사용 장치: cuda


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [8]:
import pandas as pd
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm

# ==========================================
# 1. 설정 및 데이터 로드
# ==========================================
MODEL_PATH = "./final_model_bigbird"       # 학습된 모델 경로
TEST_FILE = "/content/drive/MyDrive/DLThon/data/test.csv"                     # [수정] 업로드한 CSV 파일명

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 장치: {device}")

# 테스트 데이터 로드 (CSV)
try:
    # [수정] json.load 대신 pd.read_csv 사용
    test_df = pd.read_csv(TEST_FILE)
    print(f"테스트 데이터 로드 완료: {len(test_df)}개")
    print(test_df.head(2)) # 데이터 확인

except FileNotFoundError:
    print(f"오류: '{TEST_FILE}' 파일을 찾을 수 없습니다. 파일이 업로드되었는지 확인해주세요.")
    # (테스트용 더미 데이터 생성)
    test_df = pd.DataFrame({
        'idx': ['t_000', 't_001'],
        'conversation': ['이거 들어봐 와 이 노래 진짜 좋다', '야 돈 내놔']
    })

# ==========================================
# 2. 모델 로드
# ==========================================
print("모델 로딩 중...")
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH).to(device)
    model.eval()
except OSError:
    print("오류: 저장된 모델을 찾을 수 없습니다. 학습 코드를 먼저 실행했는지 확인해주세요.")
    # (비상용 로드 - 실제 실행 시에는 주석 처리 추천)
    tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
    model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base", num_labels=5).to(device)

# ==========================================
# 3. 추론 (Inference)
# ==========================================
print("예측 시작...")

results = []

# [수정] DataFrame 반복문으로 변경 (JSON 방식 -> CSV 방식)
# test.csv의 컬럼명은 'idx', 'conversation' 이라고 가정합니다.
for i, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    idx = row['idx']
    text = str(row['conversation']) # 텍스트가 숫자로 인식될 경우 대비해 str 변환

    # 전처리 및 토크나이징
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # 예측값 (0~4 숫자)
    pred_label = torch.argmax(logits, dim=-1).item()

    # 결과 저장
    results.append({
        'idx': idx,
        'class': pred_label
    })

# ==========================================
# 4. 제출 파일 생성
# ==========================================
submission = pd.DataFrame(results)

# 숫자(0~4)로 제출하는 것이 맞는지, 문자열('협박 대화' 등)로 제출해야 하는지에 따라 아래 주석을 해제하세요.

# [옵션] 만약 문자로 변환해서 제출해야 한다면 아래 주석 해제
"""
label_map_reverse = {
    0: '협박 대화', 1: '갈취 대화', 2: '직장 내 괴롭힘 대화', 3: '기타 괴롭힘 대화', 4: '일반 대화'
}
submission['class'] = submission['class'].map(label_map_reverse)
"""

# 파일 저장
save_path = "submission.csv"
submission.to_csv(save_path, index=False)

print("\n" + "="*50)
print(f"제출 파일 생성 완료: {save_path}")
print("="*50)
print(submission.head())

사용 장치: cuda
테스트 데이터 로드 완료: 500개
     idx                                       conversation
0  t_000  아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...
1  t_001  우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때?  네? 제가요? ...
모델 로딩 중...


Loading weights:   0%|          | 0/203 [00:00<?, ?it/s]

예측 시작...


100%|██████████| 500/500 [00:15<00:00, 31.60it/s]


제출 파일 생성 완료: submission.csv
     idx  class
0  t_000      1
1  t_001      2
2  t_002      2
3  t_003      3
4  t_004      3


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
